Load dataset

In [1]:
import pandas
import json
import numpy as np
import ast
from tqdm import tqdm
import sqlite3

In [2]:
data = pandas.read_csv("../../data/lol_dataset/archive/match_data_version1.csv")

Show the basic fields that the data has to improve the overview over the data.

In [3]:
print(data.__dict__)

{'_is_copy': None, '_mgr': BlockManager
Items: Index(['Unnamed: 0', 'gameCreation', 'gameDuration', 'gameId', 'gameMode',
       'gameType', 'gameVersion', 'mapId', 'participantIdentities',
       'participants', 'platformId', 'queueId', 'seasonId', 'status.message',
       'status.status_code'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=108829, step=1)
FloatBlock: [ 1  2  3  7 11 12 13 14], 8 x 108829, dtype: float64
IntBlock: slice(0, 1, 1), 1 x 108829, dtype: int64
ObjectBlock: [ 4  5  6  8  9 10], 6 x 108829, dtype: object, '_item_cache': {}, '_attrs': {}}


In [ ]:
print(data.participants[0])

In [40]:
dict_properties = {}
for x in data.gameMode:
    dict_properties[x] = 0
for y in dict_properties:
    print(y)
print(data.gameMode[0])

CLASSIC
ARAM
ONEFORALL
URF
TUTORIAL_MODULE_3
TUTORIAL_MODULE_1
KINGPORO
TUTORIAL_MODULE_2
CLASSIC


Read in the champion data and have a look at it.

In [2]:
champions = json.loads(open("../../data/lol_dataset/champion.json","r").read())

Create own champion ids to improve dimensionality

In [35]:
counter = 0
championDict = champions["data"]
keyChampionDict = {}
newChampionDict = {}
iterate = championDict.keys()
for x in iterate:
    keyChampionDict[championDict[x]["key"]] = (counter,x)
    if x == "Sivir":
        print("Sivir",counter)
    elif x == "Swain":
        print("Swain",counter)
    newChampionDict[counter] = x
    counter+=1
print("Distinct champions: ",counter)
max_hero_counter = counter

Sivir 112
Swain 116
Distinct champions:  155


In [4]:
def add_team(comp,team_comp,dictionary,win):
    comp.sort()
    final_string=''
    for y in comp:
        final_string+=str(y)
    if final_string in dictionary:
        (w,x,y) = dictionary[final_string]
        if win:
            x+=1
        dictionary[final_string] = (w,x,y+1)
    else:
        if win:
            dictionary[final_string] = (team_comp,1,1)
        else:
            dictionary[final_string] = (team_comp,0,1)

In [12]:
result = sqlite3.connect("league_of_legends.db")
cursor = result.cursor()
data = []
for row in cursor.execute("SELECT desc from matches;"):
    data.append(row[0])
    

In [13]:
print(len(data))

722196


In [38]:
dict_of_teams = {}
duo=0
winrate = 0
for xi in tqdm(range(len(data))):
    x = data[xi]
    try:
        winner_array = np.zeros(shape=(max_hero_counter,1))
        loser_array = np.zeros(shape=(max_hero_counter,1))
        final_string = ''
        current_list = []
        has_cain = False
        has_khazix = False
        for i in json.loads(x):
            print(i)
            break
            own_championId = keyChampionDict[str(i["championId"])][0]
            if i["stats"]["win"] == True:
                if own_championId == 112:
                    has_cain = True
                elif own_championId == 116:
                    has_khazix = True
                winner_array[own_championId] = 1
                current_list.append(own_championId)
                if len(current_list) == 5:
                    add_team(current_list,winner_array, dict_of_teams,True)
                    current_list = []
            else:
                if own_championId == 112:
                    has_cain = True
                elif own_championId == 116:
                    has_khazix = True
                loser_array[own_championId] = 1
                current_list.append(own_championId)
                if len(current_list) == 5:
                    add_team(current_list,loser_array, dict_of_teams,False)
                    current_list = []
        break
        if has_cain and has_khazix:
            duo+=1
    except KeyError:
        pass
print(duo)

  0%|          | 0/722196 [00:00<?, ?it/s]

{'championId': 360, 'stats': {'win': False}}
0


In [46]:
dict_of_teams = {}
for xi in tqdm(range(len(data.participants))):
    if data.gameMode[xi] != 'CLASSIC':
        continue
    x = data.participants[xi]
    try:
        winner_array = np.zeros(shape=(max_hero_counter,1))
        loser_array = np.zeros(shape=(max_hero_counter,1))
        final_string = ''
        current_list = []
        for i in ast.literal_eval(x):
            own_championId = keyChampionDict[str(i["championId"])][0]
            if i["stats"]["win"] == True:
                winner_array[own_championId] = 1
                current_list.append(own_championId)
                if len(current_list) == 5:
                    add_team(current_list,winner_array, dict_of_teams,True)
                    current_list = []
            else:
                loser_array[own_championId] = 1
                current_list.append(own_championId)
                if len(current_list) == 5:
                    add_team(current_list,loser_array, dict_of_teams,False)
                    current_list = []
    except KeyError:
        pass

100%|██████████| 108829/108829 [09:36<00:00, 188.78it/s]


In [19]:
counter = 0
most_played = []
for x in dict_of_teams.values():
    counter+=x[2]
    if x[2] > 5:
        most_played.append(x)
print(counter)
        
most_played.sort(key=lambda x: x[2],reverse=True)
counter = 0
for y in most_played:
    counter+=1
    result = np.where(y[0] == 1)[0]
    print("Played ",y[1],"/",y[2]," times: ",list(map(lambda x: newChampionDict[x], result)))
print(counter)

1443814
Played  119 / 4920  times:  ['Alistar', 'Ezreal', 'Galio', 'Nasus', 'Ryze']
Played  468 / 893  times:  ['Lux']
Played  475 / 816  times:  ['Yasuo']
Played  437 / 777  times:  ['Yone']
Played  358 / 720  times:  ['Gwen']
Played  433 / 693  times:  ['Jhin']
Played  409 / 667  times:  ['LeeSin']
Played  286 / 603  times:  ['Shaco']
Played  247 / 582  times:  ['Ezreal']
Played  213 / 573  times:  ['Blitzcrank']
Played  266 / 555  times:  ['Zed']
Played  228 / 554  times:  ['Morgana']
Played  337 / 552  times:  ['Malphite']
Played  199 / 510  times:  ['Pyke']
Played  261 / 470  times:  ['Veigar']
Played  262 / 456  times:  ['Viego']
Played  246 / 451  times:  ['Sylas']
Played  213 / 446  times:  ['Akali']
Played  191 / 421  times:  ['Caitlyn']
Played  238 / 421  times:  ['Ashe']
Played  236 / 420  times:  ['Senna']
Played  263 / 402  times:  ['Chogath']
Played  253 / 401  times:  ['MissFortune']
Played  126 / 383  times:  ['Rammus']
Played  201 / 379  times:  ['Darius']
Played  219 